In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [11]:
import torch
import torch.nn as nn
import pandas as pd
from data_process import *
from data_split import *
from torch.utils.data import Dataset, DataLoader

In [5]:
def mygenerate_data(root_path, by_txt=True, shuffle=True, factor=0.2, snr=5, resample=True):
    """
    根据打好标签的 txt 文件导入数据，并按文件来划分训练集以及测试集
    其中训练集，测试集默认按 0.8 0.2 比例划分
    数据集目录结构: area/data/, area/txt/
    """
    data_root, txt_root = root_path + '/data', root_path + '/txt'
    train_data, test_data = [], []
    file_data_dict = {}

    file_name_list = os.listdir(data_root)

    for file_name in file_name_list:
        
        file_path = data_root + '/' + file_name
        
        dataXYZ = pd.read_csv(file_path, header= 0)

        data_x, data_y, data_z = list(dataXYZ.iloc[:,0]), list(dataXYZ.iloc[:, 1]), list(dataXYZ.iloc[:, 2])
        
        if resample:
            data_x, data_y, data_z = data_resample(data_x, 2, 1), data_resample(data_y, 2, 1), \
                data_resample(data_z, 2, 1)
        
        dataXYZ = pd.DataFrame()
        dataXYZ['x'] = data_x
        dataXYZ['y'] = data_y
        dataXYZ['z'] = data_z

        base_value = cal_base_value(dataXYZ, 16, 8, 200)
        
        if by_txt:
            txt_path = txt_root + '/' + file_name[:-3] + 'txt'
            with open(txt_path, 'r') as f:
                activity_list = f.readlines()
            activity_list = [int(activity[:-1]) for activity in activity_list]
        else:
            activity_list = [int(np.mean(idx)) for idx in activitySplit(dataXYZ, 16, 8, 200)]

        new_list = []
        for center in activity_list:
            item = {'data_x': np.array(extract_data_from_center(data_x, center, base_value[0], length=64)),
                    'data_y': np.array(extract_data_from_center(data_y, center, base_value[1], length=64)),
                    'data_z': np.array(extract_data_from_center(data_z, center, base_value[2], length=64)),
                    'label': get_activity_label(file_name), 'file_name': file_name, 'base_value':base_value,
                    'angle': cal_angles(base_value), 'area': get_area_label(root_path) }
            
            noise_z = np.array(extract_data_from_center(data_z, center+32, base_value[2], 64))
            item['snr'] = cal_snr(item['data_z']-item['base_value'][2], noise_z-item['base_value'][2])

            
            new_list.append(item)
        # activity_list = [{'data_x': np.array(extract_data_from_center(data_x, center, base_value[0])),
        #                 'data_y': np.array(extract_data_from_center(data_y, center, base_value[1])),
        #                 'data_z': np.array(extract_data_from_center(data_z, center, base_value[2])),
        #                 'label': get_activity_label(file_name), 'file_name': file_name, 'base_value':base_value,
        #                 'angle': cal_angles(base_value), 'area': get_area_label(root_path) }
        #                 for center in activity_list]
        
        file_data_dict[file_name] = filter_by_snr(new_list, snr)

        if shuffle:
            random.shuffle(new_list)
        
        test_data = test_data + new_list[: int(factor * len(new_list))]
        train_data = train_data + new_list[int(factor * len(new_list)): ]
        
    return filter_by_snr(train_data, snr), filter_by_snr(test_data, snr), file_name_list, file_data_dict

In [6]:
def sig_diff(sig):
    return [sig[1]-sig[0]] + [sig[i+1] - sig[i] for i in range(len(sig)-1)]

def newhandle_data_3dims(item, mode='origin'):
    """
    将单个切割出来的数据处理按mode处理成三轴数
    mode: 'origin'-只减基线，'combine'-转换为x2+y2+z2, x2+y2, z三轴数据
    """
    base, angle = item['base_value'], item['angle'] # xyz的基线，以及其与g的夹角
    data_x, data_y, data_z = item['data_x'], item['data_y'], item['data_z']
    var = [0.00076385, 0.00017194, 0.00071417, 0.000022871, 0.000040234]

    if mode == 'combine':
        data_xyz = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2 + (data_z-base[2])**2) # x2+y2+z2不论如何都减基线
        data_z_rectify = (data_x-base[0]) * angle['x'] + (data_y-base[1]) * angle['y'] + (data_z-base[2]) * angle['z']
        data_xy = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2)
        
        data = np.array([cut_mean(data_xyz) / np.sqrt(var[0]), cut_mean(data_xy) / np.sqrt(var[1]), \
            cut_mean(data_z_rectify) / np.sqrt(var[2])], dtype=np.float64)

    elif mode == 'origin':
        data = np.array([data_x-base[0], data_y-base[1], data_z-base[2]], dtype=np.float64)

    elif mode == 'all':
        data_xyz = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2 + (data_z-base[2])**2) # x2+y2+z2不论如何都减基线
        data_z_rectify = (data_x-base[0]) * angle['x'] + (data_y-base[1]) * angle['y'] + (data_z-base[2]) * angle['z']
        data_xy = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2)

        data = np.array([cut_mean(data_xyz) / np.sqrt(var[0]), \
            cut_mean(data_xy) / np.sqrt(var[1]), \
            cut_mean(data_z_rectify) / np.sqrt(var[2]), \
            cut_mean(data_x) / np.sqrt(var[3]), \
            cut_mean(data_y) / np.sqrt(var[4])], dtype=np.float64)

    elif mode == 'diff':
        diff_x, diff_y, diff_z = sig_diff(data_x), sig_diff(data_y), sig_diff(data_z)
        data = np.array([diff_x, diff_y, diff_z])

    elif mode == 'diff2':
        data_xyz = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2 + (data_z-base[2])**2) # x2+y2+z2不论如何都减基线
        data_z_rectify = (data_x-base[0]) * angle['x'] + (data_y-base[1]) * angle['y'] + (data_z-base[2]) * angle['z']
        data_xy = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2)

        diff_xyz, diff_g, diff_xy = sig_diff(data_xyz), sig_diff(data_z_rectify), sig_diff(data_xy)

        data = np.array([diff_xyz, diff_g, diff_xy])

    elif mode == 'diff3':
        data_xyz = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2 + (data_z-base[2])**2) # x2+y2+z2不论如何都减基线
        data_z_rectify = (data_x-base[0]) * angle['x'] + (data_y-base[1]) * angle['y'] + (data_z-base[2]) * angle['z']
        data_xy = np.sqrt((data_x-base[0])**2 + (data_y-base[1])**2)

        diff_xyz, diff_g, diff_xy = sig_diff(data_xyz), sig_diff(data_z_rectify), sig_diff(data_xy)

        data = np.array([diff_xyz, diff_g, diff_xy, \
            cut_mean(data_xyz) / np.sqrt(var[0]), \
            cut_mean(data_xy) / np.sqrt(var[1]), \
            cut_mean(data_z_rectify) / np.sqrt(var[2])])

    else:
        raise ValueError("Unrecognized mode: {}".format(mode))
    
    return data

def newhandle_dataset_3dims(dataset, mode='origin'):
    """
    对原始的数据进行处理，生成 data 与对应的 label
    file_name_list: 需要用于生成数据集的文件名，在测试时可以选择几个文件单独生成数据集
    mode: 'origin'-只减基线，'combine'-转换为x2+y2+z2, x2+y2, z三轴数据
    """
    
    data = []
    label = []

    for item in dataset:
        data.append(newhandle_data_3dims(item, mode))
        label.append(item['label'])
    
    data = np.array(data, dtype=np.float64)
    label = np.array(label)
    return data, label

In [7]:
snr=0
print("generating data...")
syf_train, syf_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/syf', by_txt=False, snr=snr)
syf2_train, syf2_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/syf2', by_txt=False, snr=snr)

yqcc_train, yqcc_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/yqcc2', by_txt=False, snr=snr)
yqcc2_train, yqcc2_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/yqcc2_md', by_txt=False, snr=snr)

zwy_train, zwy_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zwy', by_txt=False, snr=snr)
zwy2_train, zwy2_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zwy_d1', by_txt=False, snr=snr)
zwy3_train, zwy3_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zwy_418', by_txt=False, snr=snr)
zwy4_train, zwy4_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zwy_423', by_txt=False, snr=snr)
zwy5_train, zwy5_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zwy_621', by_txt=False, snr=snr)

j11_train, j11_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/j11', by_txt=False, snr=snr)
j11_2_train, j11_2_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/j11_328', by_txt=False, snr=snr)
j11_md_train, j11_md_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/j11_49', by_txt=False, snr=snr)
j11_527_train, j11_527_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/j11_527', by_txt=False, snr=snr)

zyq_train, zyq_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zyq', by_txt=False, snr=snr)
zyq2_train, zyq2_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/zyq_d1', by_txt=False, snr=snr)

j7lqc_train, j7lqc_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/j7lqc', by_txt=False, snr=snr)

sky_train, sky_test, _, _ = mygenerate_data('D:/研一/嗑盐/土壤扰动/dataset/sky', by_txt=False, snr=snr)
print("generating data finishing...")

generating data...
generating data finishing...


In [8]:
train_data = syf_train + syf2_train + yqcc_train + yqcc2_train + zwy_train + zwy2_train + zwy3_train + zwy4_train + \
    zwy5_train + j11_train + j11_2_train + j11_md_train + j11_527_train + j7lqc_train + sky_train

test_data = syf_test + syf2_test + yqcc_test + yqcc2_test + zwy_test + zwy2_test + zwy3_test + zwy4_test + \
    zwy5_test + j11_test + j11_2_test + j11_md_test + j11_527_test + j7lqc_test + sky_test

# train_data = zwy_train + zwy2_train + zwy3_train + zwy4_train
# test_data = zwy_test + zwy2_test + zwy3_test + zwy4_test

random.shuffle(train_data)
random.shuffle(test_data)

In [9]:
train_x, train_label = newhandle_dataset_3dims(train_data, mode="all")
test_x, test_label = newhandle_dataset_3dims(test_data, mode="all")

In [20]:
train_x = np.swapaxes(train_x, 2, 1)
test_x = np.swapaxes(test_x, 2, 1)
print(train_x.shape, train_label.shape)

(19389, 64, 5) (19389,)


In [35]:
a = np.array([[[1, 2, 3], [3, 4, 5]], [[6, 2, 3], [3, 4, 7]]])
print(a.shape)
np.max(a.reshape(-1, a.shape[-1]), axis=0)

(2, 2, 3)


array([6, 4, 7])

In [36]:
class tsMinMaxNormlizer:
    "用于对dataframe型的序列做最大最小归一化"
    def __init__(self, scale=(0, 1)):
        self.scale = scale

    def fit(self, X):

        self.data_max_ = np.max(X.reshape(-1, X.shape[-1]), axis=0)
        self.data_min_ = np.min(X.reshape(-1, X.shape[-1]), axis=0)

    def transform(self, x):
        # 输入x为numpy.array, x shape: (seq_len, dim)
        result = []
        for i in range(x.shape[-1]):
            _x = x[:, i]
            _x = (_x - self.data_min_[i]) / (self.data_max_[i] - self.data_min_[i])
            _x = self.scale[0] + _x * (self.scale[1] - self.scale[0])
            result.append(_x[:, np.newaxis])
        
        return np.concatenate(result, axis=-1)

In [63]:
norm = tsMinMaxNormlizer(scale=(0.05, 0.95))
norm.fit(train_x)

In [48]:
from dltime.data.ts_datasets import noise_mask
class MLM_Soil_Dataset(Dataset):
    "Torch Datasets for UCR/UEA archive"

    def __init__(self, data, add_cls=True, pt_ratio=0.5, max_len=128, normalize=None, \
        masking_ratio=0.2, lm=5, mode='separate', distribution='geometric'):

        assert normalize in ["standard", "minmax", None]

        super().__init__()
        self.data = data
        self.add_cls = add_cls
        self.pt_ratio = pt_ratio
        self.max_len = max_len
        self.normalize = normalize
        self.masking_ratio = masking_ratio
        self.lm = lm
        self.mode = mode
        self.distribution = distribution
        self.normalizer = tsMinMaxNormlizer(scale=(0.05, 0.95))
        self.normalizer.fit(self.data)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = self.data[idx]

        # 数据归一化, 均按维度进行归一化
        X = self.normalizer.transform(X)
        # padding mask
        padding_mask = [0] + [0] * X.shape[0] + [1] * (self.max_len - X.shape[0] - 1)
        # lm mask
        lm_mask = ~noise_mask(X, self.masking_ratio, self.lm, self.mode, self.distribution)

        cls = np.ones((1, X.shape[-1])) # [CLS]
        pad = np.zeros((self.max_len - X.shape[0] - 1, X.shape[-1])) # [PAD]
        X = np.concatenate([cls, X, pad], axis=0)
        
        # lm_mask
        cls_mask = np.zeros((1, X.shape[-1]), dtype=np.bool) # [CLS]
        pad_mask = pad[:]
        lm_mask = torch.from_numpy(np.concatenate([cls_mask, lm_mask, pad_mask], axis=0)).bool()

        item = {"input": torch.from_numpy(X[:]).masked_fill(lm_mask, -1).float(), \
            "padding_mask": torch.tensor(padding_mask).bool(), 
            "output": torch.from_numpy(X[:]).float(), 
            "lm_mask": lm_mask}
        
        return item

In [49]:
train_mlm_dataset = MLM_Soil_Dataset(train_x)
test_mlm_dataset = MLM_Soil_Dataset(test_x)

Pretrain

In [74]:
import time
import gc
from dltime.models.ts_transformer import TSTransformerEncoderMLM
from config import TrainConfig
from utils import get_logger, get_scheduler
from transformers import AdamW
from train_helper import mlm_train_fn, mlm_valid_fn

In [78]:
CFG = TrainConfig()
CFG.encoder_lr, CFG.decoder_lr = 2e-3, 2e-3

In [67]:
train_mlm_dataloader = DataLoader(train_mlm_dataset, batch_size=CFG.batch_size, shuffle=True)
test_mlm_dataloader = DataLoader(test_mlm_dataset, batch_size=CFG.batch_size)

In [69]:
LOGGER = get_logger("soil_mlm_train")
LOGGER.info(f'========= Training =========')

========= Training =========


In [76]:
feat_dim = train_x.shape[-1]
max_len = 128
model = TSTransformerEncoderMLM(
    feat_dim=feat_dim, 
    max_len=max_len,
    d_model=64, n_heads=2, num_layers=2, 
    dim_feedforward=512).to(CFG.device)

In [72]:
optimizer_parameters = model.parameters()
    # optimizer_parameters = get_optimizer_params(model, CFG.encoder_lr, CFG.decoder_lr)
optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

num_train_steps = int(len(train_mlm_dataset) / CFG.batch_size * CFG.epochs)
scheduler = get_scheduler(CFG, optimizer, num_train_steps)

criterion = nn.MSELoss(reduction="none")
best_score = 0.

C:\Software\anaconda3\envs\torch\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [79]:
for epoch in range(CFG.epochs):

    start_time = time.time()

    # train
    avg_loss = mlm_train_fn(CFG, train_mlm_dataloader, model, criterion, optimizer, epoch, scheduler, CFG.device)

    # eval
    avg_val_loss = mlm_valid_fn(CFG, test_mlm_dataloader, model, criterion, CFG.device)

    elapsed = time.time() - start_time

    LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    
    if best_score > avg_val_loss:
        best_score = avg_val_loss
        LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        torch.save(model.state_dict(), f"outputs/soil_best.pth")

torch.cuda.empty_cache()
gc.collect()

Epoch: [1][0/1212] Elapsed 0m 0s (remain 0m 49s) Loss: 1.0210(1.0210) Grad: 1021.0754  LR: 0.00009987  
Epoch: [1][100/1212] Elapsed 0m 1s (remain 0m 18s) Loss: 1.1395(1.0532) Grad: 1011.2743  LR: 0.00009986  
Epoch: [1][200/1212] Elapsed 0m 3s (remain 0m 16s) Loss: 1.0538(1.0558) Grad: 1010.4191  LR: 0.00009985  
Epoch: [1][300/1212] Elapsed 0m 4s (remain 0m 14s) Loss: 1.1217(1.0507) Grad: 1010.1683  LR: 0.00009984  
Epoch: [1][400/1212] Elapsed 0m 6s (remain 0m 12s) Loss: 1.0660(1.0523) Grad: 1010.5606  LR: 0.00009982  
Epoch: [1][500/1212] Elapsed 0m 7s (remain 0m 10s) Loss: 1.0106(1.0514) Grad: 1010.7153  LR: 0.00009981  
Epoch: [1][600/1212] Elapsed 0m 9s (remain 0m 9s) Loss: 1.0564(1.0551) Grad: 1009.8890  LR: 0.00009980  
Epoch: [1][700/1212] Elapsed 0m 10s (remain 0m 7s) Loss: 0.9709(1.0548) Grad: 1010.2654  LR: 0.00009979  
Epoch: [1][800/1212] Elapsed 0m 12s (remain 0m 6s) Loss: 1.1066(1.0558) Grad: 1011.2684  LR: 0.00009978  
Epoch: [1][900/1212] Elapsed 0m 13s (remain 0m 4s

KeyboardInterrupt: 